## WWC ML for Social Good Covid-19 Hackathon

### Household Pulse Survey and Mental Health

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns
import random

import boto3
import io
import os
import sagemaker.amazon.common as smac 
import sagemaker
from sagemaker import get_execution_role

In [6]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
# print full data frame

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
# only run when using AWS
role = get_execution_role()
input_bucket = "pythonpings"
in_data_key = "one_hot.csv"
training_data_location = 's3://{}/{}'.format(input_bucket, in_data_key)

In [50]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} 
# each region has its XGBoost container

### Importing One Hot Data

In [32]:
data = pd.read_csv(training_data_location)

In [33]:
data.head(10)

,Unnamed: 0,eeduc,curfoodsuf,childfood,tspndfood,tspndprpd,foodconf,hlthstatus,anxious,worry,interest,down,mortconf,compavail,intrntavail,tschlhrs,tstdy_hrs,ttch_hrs,income,age_range,egender_1,egender_2,rhispanic_1,rhispanic_2,rrace_1,rrace_2,rrace_3,rrace_4,ms_0,ms_1,ms_2,ms_3,ms_4,ms_5,thhld_numper_1,thhld_numper_2,thhld_numper_3,thhld_numper_4,thhld_numper_5,thhld_numper_6,thhld_numper_7,thhld_numper_8,thhld_numper_9,thhld_numper_10,thhld_numkid_0,thhld_numkid_1,thhld_numkid_2,thhld_numkid_3,thhld_numkid_4,thhld_numkid_5,wrkloss_0,wrkloss_1,wrkloss_2,expctloss_0,expctloss_1,expctloss_2,eip_0,eip_1,eip_2,eip_3,eip_4,anywork_0,anywork_1,anywork_2,rsnnowrk_0,rsnnowrk_1,rsnnowrk_2,rsnnowrk_3,rsnnowrk_4,rsnnowrk_5,rsnnowrk_6,rsnnowrk_7,rsnnowrk_8,rsnnowrk_9,rsnnowrk_10,rsnnowrk_11,rsnnowrk_12,rsnnowrk_13,unemppay_0,unemppay_1,unemppay_2,unemppay_3,unemppay_4,mortlmth_0,mortlmth_1,mortlmth_2,mortlmth_3,tenure_0,tenure_1,tenure_2,tenure_3,tenure_4,delay_0,delay_1,delay_2,freefood_0,freefood_1,freefood_2
0,0,7,2,0,11.111111,4.0,2,2,2,2,2,1,0,0,0,0.0,0.0,0.0,5,25-35,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1
1,1,6,1,0,2.222222,3.0,4,3,1,1,1,2,4,0,0,0.0,0.0,0.0,3,35-45,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
2,2,7,1,0,6.666667,8.0,4,3,4,4,4,4,4,0,0,0.0,0.0,0.0,8,25-35,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1
3,3,6,2,0,4.533333,0.6,2,3,1,1,1,1,4,0,0,0.0,0.0,0.0,4,65+,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
4,4,6,1,0,2.777778,0.0,4,3,1,1,1,1,0,0,0,0.0,0.0,0.0,5,65+,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1
5,5,5,2,0,0.000000,2.4,3,4,2,2,2,2,3,0,0,0.0,0.0,0.0,1,65+,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
6,6,6,1,0,1.111111,6.0,4,2,1,1,1,1,0,0,0,0.0,0.0,0.0,4,55-65,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1
7,7,7,1,0,1.111111,3.0,4,2,2,1,1,1,4,0,0,0.0,0.0,0.0,8,35-45,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
8,8,6,1,0,4.444444,0.0,4,3,1,1,2,1,0,0,0,0.0,0.0,0.0,6,65+,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1
9,9,6,1,0,6.666667,8.0,4,1,2,2,1,2,4,1,1,0.0,0.0,0.0,8,35-45,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1


In [34]:
data.drop(data.columns[0], axis=1, inplace=True)
#drop unamed id column

In [35]:
mapped = {"age_range": {"0-18": 1, 
                              "18-25": 2,
                              "25-35": 3,
                              "35-45": 4,
                              "45-55": 5,
                              "55-65": 6,
                              "65+": 7}}

In [36]:
data.replace(mapped, inplace=True)
data.head()

,eeduc,curfoodsuf,childfood,tspndfood,tspndprpd,foodconf,hlthstatus,anxious,worry,interest,down,mortconf,compavail,intrntavail,tschlhrs,tstdy_hrs,ttch_hrs,income,age_range,egender_1,egender_2,rhispanic_1,rhispanic_2,rrace_1,rrace_2,rrace_3,rrace_4,ms_0,ms_1,ms_2,ms_3,ms_4,ms_5,thhld_numper_1,thhld_numper_2,thhld_numper_3,thhld_numper_4,thhld_numper_5,thhld_numper_6,thhld_numper_7,thhld_numper_8,thhld_numper_9,thhld_numper_10,thhld_numkid_0,thhld_numkid_1,thhld_numkid_2,thhld_numkid_3,thhld_numkid_4,thhld_numkid_5,wrkloss_0,wrkloss_1,wrkloss_2,expctloss_0,expctloss_1,expctloss_2,eip_0,eip_1,eip_2,eip_3,eip_4,anywork_0,anywork_1,anywork_2,rsnnowrk_0,rsnnowrk_1,rsnnowrk_2,rsnnowrk_3,rsnnowrk_4,rsnnowrk_5,rsnnowrk_6,rsnnowrk_7,rsnnowrk_8,rsnnowrk_9,rsnnowrk_10,rsnnowrk_11,rsnnowrk_12,rsnnowrk_13,unemppay_0,unemppay_1,unemppay_2,unemppay_3,unemppay_4,mortlmth_0,mortlmth_1,mortlmth_2,mortlmth_3,tenure_0,tenure_1,tenure_2,tenure_3,tenure_4,delay_0,delay_1,delay_2,freefood_0,freefood_1,freefood_2
0,7,2,0,11.111111,4.0,2,2,2,2,2,1,0,0,0,0.0,0.0,0.0,5,3,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1
1,6,1,0,2.222222,3.0,4,3,1,1,1,2,4,0,0,0.0,0.0,0.0,3,4,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
2,7,1,0,6.666667,8.0,4,3,4,4,4,4,4,0,0,0.0,0.0,0.0,8,3,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1
3,6,2,0,4.533333,0.6,2,3,1,1,1,1,4,0,0,0.0,0.0,0.0,4,7,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
4,6,1,0,2.777778,0.0,4,3,1,1,1,1,0,0,0,0.0,0.0,0.0,5,7,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1


In [37]:
data.corr()

,eeduc,curfoodsuf,childfood,tspndfood,tspndprpd,foodconf,hlthstatus,anxious,worry,interest,down,mortconf,compavail,intrntavail,tschlhrs,tstdy_hrs,ttch_hrs,income,age_range,egender_1,egender_2,rhispanic_1,rhispanic_2,rrace_1,rrace_2,rrace_3,rrace_4,ms_0,ms_1,ms_2,ms_3,ms_4,ms_5,thhld_numper_1,thhld_numper_2,thhld_numper_3,thhld_numper_4,thhld_numper_5,thhld_numper_6,thhld_numper_7,thhld_numper_8,thhld_numper_9,thhld_numper_10,thhld_numkid_0,thhld_numkid_1,thhld_numkid_2,thhld_numkid_3,thhld_numkid_4,thhld_numkid_5,wrkloss_0,wrkloss_1,wrkloss_2,expctloss_0,expctloss_1,expctloss_2,eip_0,eip_1,eip_2,eip_3,eip_4,anywork_0,anywork_1,anywork_2,rsnnowrk_0,rsnnowrk_1,rsnnowrk_2,rsnnowrk_3,rsnnowrk_4,rsnnowrk_5,rsnnowrk_6,rsnnowrk_7,rsnnowrk_8,rsnnowrk_9,rsnnowrk_10,rsnnowrk_11,rsnnowrk_12,rsnnowrk_13,unemppay_0,unemppay_1,unemppay_2,unemppay_3,unemppay_4,mortlmth_0,mortlmth_1,mortlmth_2,mortlmth_3,tenure_0,tenure_1,tenure_2,tenure_3,tenure_4,delay_0,delay_1,delay_2,freefood_0,freefood_1,freefood_2
eeduc,1.000000,-0.226723,-0.138680,0.006375,-0.015069,0.267687,-0.168610,-0.014884,-0.063368,-0.085443,-0.067996,0.102642,-0.068897,-0.063587,-0.000785,0.011123,-0.002642,0.348306,0.060123,0.026737,-0.026737,0.135132,-0.135132,0.054924,-0.074842,0.067978,-0.067142,-0.008384,0.130622,-0.045649,-0.055285,-0.063450,-0.063737,0.018406,0.047213,-0.016096,0.011844,-0.032708,-0.048133,-0.039790,-0.037363,-0.022480,-0.029891,0.045962,-0.033752,0.013848,-0.032615,-0.031889,-0.037547,-0.010310,-0.130167,0.131399,-0.013950,-0.115764,0.117394,-0.013241,-0.200355,0.026247,0.095628,0.158520,-0.013347,0.117917,-0.116178,0.112913,-0.011442,-0.045248,-0.024576,-0.063081,-0.029990,-0.084069,0.032091,-0.036748,-0.051672,-0.045793,-0.024090,-0.038752,-0.048198,0.105537,-0.021626,0.009847,-0.042142,-0.096998,-0.016466,0.079989,-0.126759,-0.015954,-0.074563,0.047859,0.117576,-0.124242,-0.041662,-0.073165,0.054592,-0.010922,-0.045824,-0.101767,0.112733
curfoodsuf,-0.226723,1.000000,0.462885,0.031032,-0.001958,-0.351014,0.315132,0.296129,0.333528,0.330698,0.330426,-0.050802,0.167845,0.165548,0.043011,0.059590,0.079377,-0.210630,-0.119267,-0.044684,0.044684,-0.096438,0.096438,-0.126897,0.117336,-0.004083,0.078693,-0.074849,-0.140257,0.001081,0.083572,0.080439,0.086432,0.002122,-0.089269,0.023898,0.011369,0.042938,0.047332,0.040259,0.035899,0.022478,0.017485,-0.095248,0.049468,0.025584,0.046235,0.038686,0.047878,-0.099543,0.242699,-0.227735,-0.093416,0.272103,-0.253744,-0.104249,0.258880,-0.055413,-0.165825,-0.116566,-0.107632,-0.092670,0.108281,-0.108721,-0.027941,0.056980,0.036428,0.075231,0.039722,0.100480,-0.118000,0.078623,0.100382,0.075214,0.073211,0.054456,0.065577,-0.119324,0.015031,-0.008070,0.048428,0.109994,-0.123093,-0.026168,0.260056,0.072196,-0.088955,-0.093054,-0.057887,0.207894,0.060353,-0.091078,0.177917,-0.121210,-0.186035,0.145714,-0.026417
childfood,-0.138680,0.462885,1.000000,0.129953,0.053567,-0.193687,0.112662,0.145707,0.157880,0.133278,0.130353,0.005136,0.441166,0.436855,0.129149,0.209330,0.225801,-0.097993,-0.212046,-0.060176,0.060176,-0.093245,0.093245,-0.098179,0.085064,0.008680,0.056766,-0.016996,0.014423,-0.043192,0.003399,0.054725,-0.012762,-0.171620,-0.228233,0.072345,0.162831,0.173834,0.137435,0.099298,0.070243,0.045106,0.002493,-0.514794,0.289471,0.244446,0.177353,0.110477,0.088144,-0.021500,0.167382,-0.163915,-0.022310,0.169486,-0.164358,-0.028370,0.169852,-0.037524,-0.108294,-0.086085,-0.023029,0.018231,-0.014948,0.015326,-0.011816,0.030950,0.019311,0.159808,-0.001008,0.009748,-0.154432,0.035718,0.040798,0.032805,0.030034,0.013542,0.012337,0.001034,0.013677,-0.008659,0.027133,-0.007394,-0.106990,0.005896,0.167069,0.059548,-0.020436,-0.112362,0.026051,0.089772,0.015437,-0.020384,0.093937,-0.080090,-0.055214,0.145744,-0.096623
tspndfood,0.006375,0.031032,0.129953,1.000000,0.429951,0.152144,0.101653,0.153285,0.149315,0.121346,0.122040,0.139355,0.234682,0.233205,0.120074,0.155518,0.150900,0.241800,-0.113923,0.026204,-0.026204,-0.079324,0

In [38]:
data = data.sample(frac=1).reset_index(drop=True)
# shuffle data and drop the other index column

In [58]:
data.tail(10)

,eeduc,curfoodsuf,childfood,tspndfood,tspndprpd,foodconf,hlthstatus,anxious,worry,interest,down,mortconf,compavail,intrntavail,tschlhrs,tstdy_hrs,ttch_hrs,income,age_range,egender_1,egender_2,rhispanic_1,rhispanic_2,rrace_1,rrace_2,rrace_3,rrace_4,ms_0,ms_1,ms_2,ms_3,ms_4,ms_5,thhld_numper_1,thhld_numper_2,thhld_numper_3,thhld_numper_4,thhld_numper_5,thhld_numper_6,thhld_numper_7,thhld_numper_8,thhld_numper_9,thhld_numper_10,thhld_numkid_0,thhld_numkid_1,thhld_numkid_2,thhld_numkid_3,thhld_numkid_4,thhld_numkid_5,wrkloss_0,wrkloss_1,wrkloss_2,expctloss_0,expctloss_1,expctloss_2,eip_0,eip_1,eip_2,eip_3,eip_4,anywork_0,anywork_1,anywork_2,rsnnowrk_0,rsnnowrk_1,rsnnowrk_2,rsnnowrk_3,rsnnowrk_4,rsnnowrk_5,rsnnowrk_6,rsnnowrk_7,rsnnowrk_8,rsnnowrk_9,rsnnowrk_10,rsnnowrk_11,rsnnowrk_12,rsnnowrk_13,unemppay_0,unemppay_1,unemppay_2,unemppay_3,unemppay_4,mortlmth_0,mortlmth_1,mortlmth_2,mortlmth_3,tenure_0,tenure_1,tenure_2,tenure_3,tenure_4,delay_0,delay_1,delay_2,freefood_0,freefood_1,freefood_2
258917,7,1,0,1.555556,0.52,4,2,4,3,1,2,4,0,0,0.0,0.0,0.0,2,4,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1
258918,5,1,0,3.888889,2.00,4,4,1,1,1,1,4,0,0,0.0,0.0,0.0,3,5,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
258919,6,0,0,0.000000,4.00,1,1,1,3,2,2,2,0,0,0.0,0.0,0.0,5,5,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1
258920,4,2,0,2.222222,0.00,2,4,3,3,3,3,1,0,0,0.0,0.0,0.0,2,6,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1
258921,7,1,0,0.000000,0.00,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,7,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1
258922,6,3,0,2.222222,3.20,2,2,3,3,2,4,2,0,0,0.0,0.0,0.0,4,3,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
258923,7,1,0,3.888889,6.00,4,1,1,1,1,1,0,0,0,0.0,0.0,0.0,8,7,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1
258924,7,2,0,5.555556,2.00,2,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,5,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1
258925,7,1,0,2.222222,2.00,4,2,2,1,1,1,4,0,0,0.0,0.0,0.0,5,3,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
258926,4,1,0,5.555556,1.60,4,2,2,2,1,2,4,0,0,0.0,0.0,0.0,7,5,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1


In [40]:
data.to_csv("df.csv", header=False, index=False)

In [42]:
# split the data for training 60%, validation 20%, and test 20% 
#train_data, validation_data, test_data = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

In [43]:
# convert data frames to .csv and save to notebook instance
#train_data.to_csv('train.csv', header=False, index=False)
#validation_data.to_csv('validation.csv', header=False, index=False)
#test_data.to_csv('test.csv', header=False, index=False)

In [44]:
# load files to S3
#output_bucket = "pythonpings"
#prefix = "ready"

#boto3.resource('s3').Bucket(output_bucket).Object(os.path.join(prefix, "df.csv.csv")).upload_file("df.csv")
#boto3.resource('s3').Bucket(output_bucket).Object(os.path.join(prefix, "train.csv")).upload_file("train.csv")
#boto3.resource('s3').Bucket(output_bucket).Object(os.path.join(prefix, "validation.csv")).upload_file("validation.csv")
#boto3.resource('s3').Bucket(output_bucket).Object(os.path.join(prefix, "test.csv")).upload_file("test.csv")

In [45]:
#data.shape

(258927, 97)

In [59]:
train_data, test_data = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(data))])
print(train_data.shape, test_data.shape)

(181248, 97) (77679, 97)


In [60]:
train_data.head()

,eeduc,curfoodsuf,childfood,tspndfood,tspndprpd,foodconf,hlthstatus,anxious,worry,interest,down,mortconf,compavail,intrntavail,tschlhrs,tstdy_hrs,ttch_hrs,income,age_range,egender_1,egender_2,rhispanic_1,rhispanic_2,rrace_1,rrace_2,rrace_3,rrace_4,ms_0,ms_1,ms_2,ms_3,ms_4,ms_5,thhld_numper_1,thhld_numper_2,thhld_numper_3,thhld_numper_4,thhld_numper_5,thhld_numper_6,thhld_numper_7,thhld_numper_8,thhld_numper_9,thhld_numper_10,thhld_numkid_0,thhld_numkid_1,thhld_numkid_2,thhld_numkid_3,thhld_numkid_4,thhld_numkid_5,wrkloss_0,wrkloss_1,wrkloss_2,expctloss_0,expctloss_1,expctloss_2,eip_0,eip_1,eip_2,eip_3,eip_4,anywork_0,anywork_1,anywork_2,rsnnowrk_0,rsnnowrk_1,rsnnowrk_2,rsnnowrk_3,rsnnowrk_4,rsnnowrk_5,rsnnowrk_6,rsnnowrk_7,rsnnowrk_8,rsnnowrk_9,rsnnowrk_10,rsnnowrk_11,rsnnowrk_12,rsnnowrk_13,unemppay_0,unemppay_1,unemppay_2,unemppay_3,unemppay_4,mortlmth_0,mortlmth_1,mortlmth_2,mortlmth_3,tenure_0,tenure_1,tenure_2,tenure_3,tenure_4,delay_0,delay_1,delay_2,freefood_0,freefood_1,freefood_2
20048,4,2,3,4.444444,1.6,2,4,2,2,2,2,2,1,1,0.0,5.0,4.166667,3,4,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1
141863,7,1,0,3.333333,2.0,3,3,3,3,2,2,0,0,0,0.0,0.0,0.000000,2,3,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
258636,4,2,0,4.444444,0.0,4,2,4,4,4,2,4,0,0,0.0,0.0,0.000000,5,4,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
189166,7,1,0,6.666667,2.0,4,1,1,1,1,1,4,1,1,0.0,0.0,0.000000,8,4,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
87226,4,1,0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0,2,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1


In [61]:
len(train_data.columns)

97

In [68]:
attributes = ["anxious", "eeduc", "curfoodsuf", "childfood", "tspndfood",
              "tspndprpd", "foodconf", "hlthstatus", "worry", "interest", "down",
              "mortconf", "compavail", "intrntavail", "tschlhrs", "tstdy_hrs", "ttch_hrs",
              "income", "age_range", "egender_1", "egender_2", "rhispanic_1", "rhispanic_2",
              "rrace_1", "rrace_2", "rrace_3", "rrace_4", "ms_0", "ms_1", "ms_2", "ms_3",
              "ms_4", "ms_5","thhld_numper_1", "thhld_numper_2","thhld_numper_3","thhld_numper_4", 
              "thhld_numper_5","thhld_numper_6","thhld_numper_7","thhld_numper_8","thhld_numper_9",
              "thhld_numper_10","thhld_numkid_0","thhld_numkid_1","thhld_numkid_2","thhld_numkid_3",
              "thhld_numkid_4","thhld_numkid_5","wrkloss_0","wrkloss_1","wrkloss_2","expctloss_0",
              "expctloss_1","expctloss_2","eip_0","eip_1","eip_2","eip_3","eip_4",
              "anywork_0", "anywork_1","anywork_2","rsnnowrk_0","rsnnowrk_1","rsnnowrk_2","rsnnowrk_3", 
              "rsnnowrk_4", "rsnnowrk_5","rsnnowrk_6","rsnnowrk_7","rsnnowrk_8","rsnnowrk_9","rsnnowrk_10",
              "rsnnowrk_11","rsnnowrk_12", "rsnnowrk_13","unemppay_0","unemppay_1","unemppay_2","unemppay_3", 
              "unemppay_4","mortlmth_0","mortlmth_1","mortlmth_2","mortlmth_3","tenure_0","tenure_1","tenure_2",
              "tenure_3","tenure_4","delay_0", "delay_1", "delay_2","freefood_0","freefood_1","freefood_2"]
len(attributes)

97

In [69]:
train_data = train_data[attributes]

In [70]:
train_data.to_csv('train.csv', index=False, header=False)

In [72]:
boto3.Session().resource('s3').Bucket(input_bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(input_bucket, prefix), content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [73]:
my_region = boto3.session.Session().region_name 

In [74]:
output_location = 's3://{}/{}/output'.format(output_bucket, prefix)

In [75]:
sess = sagemaker.Session()

In [76]:
xgb = sagemaker.estimator.Estimator(containers[my_region],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.c4.xlarge',
                                    output_path=output_location,
                                    sagemaker_session=sess)
xgb.set_hyperparameters(eta=0.06,
                        alpha=0.8,
                        lambda_bias=0.8,
                        gamma=50,
                        min_child_weight=6,
                        subsample=0.5,
                        silent=0,
                        early_stopping_rounds=5,
                        objective='reg:linear',
                        num_round=1000)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [77]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(input_bucket, prefix), content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [79]:
xgb.fit({"train": s3_input_train})

2020-08-12 22:21:26 Starting - Starting the training job...
2020-08-12 22:21:28 Starting - Launching requested ML instances......
2020-08-12 22:22:33 Starting - Preparing the instances for training......
2020-08-12 22:23:33 Downloading - Downloading input data...
2020-08-12 22:24:22 Training - Training image download completed. Training in progress.
2020-08-12 22:24:22 Uploading - Uploading generated training modelArguments: train
[2020-08-12:22:24:17:INFO] Running standalone xgboost training.
[2020-08-12:22:24:17:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-08-12:22:24:17:INFO] File size need to be processed in the node: 37.76mb. Available memory size in the node: -20.36mb
[2020-08-12:22:24:17:ERROR] Customer Error: Insufficient memory available in the instance. External memory for csv input not supported.                                 Switch to larger instance with more RAM/distributed mode for training.
Traceback (most recent call last):
  File "/opt/amazon/lib/p

UnexpectedStatusException: Error for Training job xgboost-2020-08-12-22-21-26-303: Failed. Reason: ClientError: Insufficient memory available in the instance. External memory for csv input not supported.                                 Switch to larger instance with more RAM/distributed mode for training.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')